In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from typing import NamedTuple

import numpyro 
import numpyro.distributions as dist

from numpyro.handlers import condition, reparam, seed, trace
from numpyro.infer.reparam import LocScaleReparam, TransformReparam

/export/home/vgiusepp/miniconda3/envs/sbi_ds/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from math import pi

from typing import Optional, Tuple, Callable, Union, List
from functools import partial

import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from jax import vmap, jit, pmap
from jax import random

# jax.config.update("jax_enable_x64", True)

import numpy as np
from astropy import units as u
from astropy import constants as c

import odisseo
from odisseo import construct_initial_state
from odisseo.integrators import leapfrog
from odisseo.dynamics import direct_acc, DIRECT_ACC, DIRECT_ACC_LAXMAP, DIRECT_ACC_FOR_LOOP, DIRECT_ACC_MATRIX
from odisseo.option_classes import SimulationConfig, SimulationParams, MNParams, NFWParams, PlummerParams, MN_POTENTIAL, NFW_POTENTIAL
from odisseo.initial_condition import Plummer_sphere, ic_two_body, sample_position_on_sphere, inclined_circular_velocity, sample_position_on_circle, inclined_position
from odisseo.utils import center_of_mass
from odisseo.time_integration import time_integration
from odisseo.units import CodeUnits
from odisseo.visualization import create_3d_gif, create_projection_gif, energy_angular_momentum_plot
from odisseo.potentials import MyamotoNagai, NFW
from odisseo.option_classes import DIFFRAX_BACKEND, DOPRI5, TSIT5, SEMIIMPLICITEULER, LEAPFROGMIDPOINT, REVERSIBLEHEUN


plt.rcParams.update({
    'font.size': 20,
    'axes.labelsize': 20,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 15,
})

In [3]:
from jax.scipy.integrate import trapezoid
from galpy.potential import MiyamotoNagaiPotential

# Functions to compute the mass of the Myamoto-Nagai and NFW potentials
@partial(jit, static_argnames=("code_units",))
def mass_enclosed_MN(R, z, params, code_units):
    """
    Compute the mass of the Myamoto-Nagai potential at a given radius R.

    """
    a = params.MN_params.a * code_units.code_length.to(u.kpc)
    b = params.MN_params.b * code_units.code_length.to(u.kpc)
    M = params.MN_params.M * code_units.code_mass.to( u.Msun)

    # Compute the mass using the Myamoto-Nagai formula
    mp = MiyamotoNagaiPotential(amp = M, a=a, b=b,) 

    return mp.mass(R, z) * u.Msun.to(code_units.code_mass)
    
@jit
def mass_enclosed_NFW(R, params):
    """
    Compute the mass of the NFW potential at a given radius R.

    ref: wikipedia
    """
    c = params.NFW_params.c
    Mvir = params.NFW_params.Mvir
    r_s = params.NFW_params.r_s
    rho_0 = (Mvir / (4*jnp.pi * r_s**3)) * (jnp.log(1+c) - c/(1+c))**-1

    return 4*jnp.pi*rho_0*r_s**3 * (jnp.log(1 + R/r_s) - R/(r_s + R))


@partial(jit, static_argnames=("code_units",))
def to_observable(state, code_units):

    """
    Convert the state vector to observable quantities.
    The return is a 6D vector with the following elements:
    - parallax
    - b (galactic latitude)
    - l (galactic longitude)
    - v_r (radial velocity)
    - mu_b (proper motion in b direction)
    - mu_l (proper motion in l direction)
    """
    X = state[:, 0] * code_units.code_length.to(u.kpc)
    X_sun = X.at[:, 0].set(8. - X[:, 0] )
    r = jnp.linalg.norm(X_sun, axis=1)
    parallax = 1 / r
    b = jnp.arcsin(X_sun[:, 2] / r)
    l = jnp.arctan2(X_sun[:, 1], X_sun[:, 0])

    state = state.at[:, 1].set(state[:, 1]* code_units.code_velocity.to(u.km/u.s)) 
    v_x, v_y, v_z = state[:, 1, 0], state[:, 1, 1], state[:, 1, 2]
    v_l = -v_x * jnp.sin(l) + v_y * jnp.cos(l)
    v_b = -v_x * jnp.cos(l) * jnp.sin(b) - v_y * jnp.sin(l) * jnp.sin(b) + v_z * jnp.cos(b)
    v_r = v_x * jnp.cos(l) * jnp.cos(b) + v_y * jnp.sin(l) * jnp.cos(b) + v_z * jnp.sin(b)
    mu_l = v_l / (4.74047 * r)
    mu_b = v_b / (4.74047 * r)
    return jnp.stack([parallax, b, l, v_r, mu_b, mu_l], axis=1 )

    

In [4]:
code_length = 10.0 * u.kpc
code_mass = 1e4 * u.Msun
G = 1 
code_units = CodeUnits(code_length, code_mass, G=G)


In [5]:
def run_simulation(key,
                   sigma: jnp.ndarray,
                   with_noise: bool = True,
                   ):

    #config param, this cannot be differentiate 
    config = SimulationConfig(N_particles = 10_000, 
                          return_snapshots = False, 
                          num_timesteps = 1000, 
                          external_accelerations=(NFW_POTENTIAL, MN_POTENTIAL,  ), 
                          acceleration_scheme = DIRECT_ACC_MATRIX,
                          softening = (0.1 * u.kpc).to(code_units.code_length).value) #default values
    
    # simulation parameters to be sampled 
    t_end = numpyro.sample("t_end", dist.Uniform(0.500, 10.0))
    Mtot_plummer = numpyro.sample("Mtot_plummer", dist.Uniform(1e3, 1e5))
    a_plummer = numpyro.sample("a_plummer", dist.Uniform(0.1, 2.0))
    Mtot_MN = numpyro.sample("M_MN", dist.Uniform(5e10, 1e11))
    a = numpyro.sample("a", dist.Uniform(1.0, 5.0))
    b = numpyro.sample("b", dist.Uniform(0.1, 1))
    Mtot_NFW = numpyro.sample("M_NFW", dist.Uniform(5e11, 1.5e12))
    r_s = numpyro.sample("r_s", dist.Uniform(1, 20.0))

    params = SimulationParams(t_end = t_end * u.Myr.to(code_units.code_time),  
                          Plummer_params= PlummerParams(Mtot=Mtot_plummer * u.Msun.to(code_units.code_mass),
                                                        a=a_plummer * u.kpc.to(code_units.code_length)),
                           MN_params= MNParams(M=Mtot_MN * u.Msun.to(code_units.code_mass),
                                              a = a * u.kpc.to(code_units.code_length),
                                              b = b * u.kpc.to(code_units.code_length)),
                          NFW_params= NFWParams(Mvir=Mtot_NFW * u.Msun.to(code_units.code_mass),
                                               r_s= r_s * u.kpc.to(code_units.code_length),
                                               c = 8.0),                           
                          G=G, ) 
    
    # initial conditions
    #set up the particles in the initial state
    positions, velocities, mass = Plummer_sphere(key=key, params=params, config=config)

    #put the Plummer sphere in a ciruclar orbit around the NFW halo
    ra = 200*u.kpc.to(code_units.code_length)
    e = numpyro.sample("e", dist.Uniform(0.0, 0.7)) #nuance parameter 
    rp = (1-e)/(1+e) * ra
    # sample the position of the center of mass
    # Sample phi uniformly in [0, 2π]
    phi = numpyro.sample("phi", dist.Uniform(0, 2*pi)) #nuance parameter
    
    # Sample cos(theta) uniformly in [-1, 1] to ensure uniform distribution on the sphere
    costheta = numpyro.sample("costheta", dist.Uniform(-1, 1)) #nuance parameter
    theta = jnp.arccos(costheta)  # Convert to theta
    
    # Convert to Cartesian coordinates
    x = rp * jnp.sin(theta) * jnp.cos(phi)
    y = rp * jnp.sin(theta) * jnp.sin(phi)
    z = rp * jnp.cos(theta)

    pos_com = jnp.stack([x, y, z], axis=-1)

    inclination = jnp.pi/2 - jnp.acos(z/rp)

    # mass1 = mass_enclosed_MN(rp, z, params, code_units) + mass_enclosed_NFW(rp, params)
    mass1 = Mtot_MN + mass_enclosed_NFW(rp, params)
    mass2 = params.Plummer_params.Mtot 
    _, bulk_velocity, _ = ic_two_body(mass1=mass1,
                                    mass2=mass2,
                                    rp=rp,
                                    e=e,
                                    params=params)
    bulk_velocity_modulus = bulk_velocity[1, 1].reshape((1))
    vel_com = inclined_circular_velocity(pos_com, bulk_velocity_modulus, inclination)

    # Add the center of mass position and velocity to the Plummer sphere particles
    positions = positions + pos_com
    velocities = velocities + vel_com

    #initialize the initial state
    initial_state = construct_initial_state(positions, velocities)

    #time integration
    final_state = time_integration(primitive_state=initial_state, mass=mass,  params=params, config=config)

    #to observable space (parallax, b, l, v_r, mu_b, mu_l), shape (config.N_particles, 6)
    final_state = to_observable(final_state, code_units)

    if with_noise is True:

        #the noise on l and mu_l needs to be corrected by the cos(b) 
        # sigma = sigma.at[2].set(sigma[2]/jnp.cos(final_state[:, 1])) 
        # sigma = sigma.at[5].set(sigma[5]/jnp.cos(final_state[:, 1]))

        #add noise to the observable 
        x = numpyro.sample("y", dist.MultivariateNormal(
            loc = final_state,
            covariance_matrix = jnp.diag(sigma),
        ))
        
    
    else:
        x = numpyro.deterministic("y", final_state)

In [ ]:
def get_samples_and_scores(
    model,
    key,
    batch_size=1,
    score_type="density",
    thetas=None,
    with_noise=True,
):
    """Handling function sampling and computing the score from the model.

    Parameters
    ----------
    model : numpyro model
    key : PRNG Key
    batch_size : int, optional
        size of the batch to sample, by default 64
    score_type : str, optional
        'density' for nabla_theta log p(theta | y, z) or
        'conditional' for nabla_theta log p(y | z, theta), by default 'density'
    thetas : Array (batch_size, 2), optional
        thetas used to sample simulations or
        'None' sample thetas from the model, by default None
    with_noise : bool, optional
        add noise in simulations, by default True
        note: if no noise the score is only nabla_theta log p(theta, z)
        and log_prob log p(theta, z)

    Returns
    -------
    Array
        (log_prob, sample), score
    """

    params_name = ["t_end", "Mtot_plummer", "a_plummer", "M_MN", "a", "b", "M_NFW", "r_s"]

    def log_prob_fn(theta, key):
        cond_model = seed(model, key)
        cond_model = condition(
            cond_model,
            {
                "t_end": theta[0],
                "Mtot_plummer": theta[1],
                "a_plummer": theta[2],
                "M_MN": theta[3],
                "a": theta[4],
                "b": theta[5],
                "M_NFW": theta[6],
                "r_s": theta[7],
            },
        )
        model_trace = trace(cond_model).get_trace()
        sample = {
            "theta": jnp.stack(
                [model_trace[name]["value"] for name in params_name], axis=-1
            ),
            "y": model_trace["y"]["value"],
        }

        if score_type == "density":
            logp = 0
            for name in params_name:
                logp += model_trace[name]["fn"].log_prob(model_trace[name]["value"])
        elif score_type == "conditional":
            logp = 0

        if with_noise:
            logp += (
                model_trace["y"]["fn"]
                .log_prob(jax.lax.stop_gradient(model_trace["y"]["value"]))
                .sum()
            )
        
        for name in ["e", "costheta", "phi"]
            logp += model_trace[name]["fn"].log_prob(model_trace[name]["value"]).sum()

        return logp, sample

    # Split the key by batch
    keys = jax.random.split(key, batch_size)

    # Sample theta from the model
    if thetas is None:

        @jax.vmap
        def get_params(key):
            with jax.checking_leaks():
                model_trace = trace(seed(model, key)).get_trace()
                thetas = jnp.stack(
                    [model_trace[name]["value"] for name in params_name], axis=-1
                )
                return thetas

        
        thetas = get_params(keys)

    return jax.vmap(jax.value_and_grad(log_prob_fn, has_aux=True))(thetas, keys)

In [15]:
model = partial(
    run_simulation,
    key = random.PRNGKey(0),
    sigma=jnp.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
    with_noise=True,
)

In [16]:
(log_prob, sample), score = get_samples_and_scores(
    model,
    key=random.PRNGKey(0),   
)

In [18]:
score.shape

(1, 8)